In [83]:
#dependencies
import pandas as pd
import math


In [84]:
#Bring in the dataset
gss = pd.io.stata.read_stata('GSS1996.dta', convert_categoricals=False)
gss.to_csv('gss_converted.csv')

In [85]:
#drop all instances where angry event was not recalled
gss=gss.dropna(subset=["chnang7"])

In [86]:
#Recode DV to dummy
gss["pray_anger"]=gss["chnang7"]
gss.loc[gss.chnang7 == 2, 'pray_anger'] = 0

In [87]:
#Recode sex to dummy, reference category = "female"
gss["sex_r"]=gss["sex"]
gss.loc[gss.sex == 1, 'sex_r'] = 0
gss.loc[gss.sex == 2, 'sex_r'] = 1

In [88]:
#Recode race to dummy, reference category = "nonwhite"
gss["race_r"]=gss["race"]
gss.loc[gss.race == 1, 'race_r'] = 0
gss.loc[gss.race > 1, 'race_r'] = 1

In [89]:
#Recode degree to dummy, reference category = "has no degree" (note that in original study, ref cat was HAS degree)
gss["degree_r"]=gss["degree"]
gss.loc[gss.degree <=3, 'degree_r'] = 0
gss.loc[gss.degree > 3, 'degree_r'] = 1

In [90]:
#Recode region to dummy, reference category = "Not from South"
gss["region_r"]=gss["region"]
gss.loc[(gss.region <=4) | (gss.region >7), 'region_r'] = 0
gss.loc[(gss.region >=5) & (gss.region <=7), 'region_r'] = 1

In [91]:
#Recode marital to new categorical variable with reference category = "single, never married" and divorced and separated categories combined (note that this differs slightly from the original study, which did not assign "0" to any value of the variable.)
#New categories:
#0=single, never married
#1=married
#2=divorced/separated
#3=widowed
gss["marital_r"]=gss["marital"]
gss.loc[(gss.marital ==5), 'marital_r'] = 0
gss.loc[(gss.marital ==4) | (gss.marital ==3), 'marital_r'] = 2
gss.loc[(gss.marital ==3), 'marital_r'] = 3

In [92]:
#Recode parent to dummy, reference category = "is not parent"
gss["childs_r"]=gss["childs"]
gss.loc[(gss.childs >0), 'childs_r'] = 1


In [93]:
#Center the age variable
gss["age_c"]=gss["age"]-gss["age"].mean()

In [94]:
#Recode fundamentalism to dummy. Reference category =  "not fundamentalist"
gss["fund_r"]=gss["fund"]
gss.loc[(gss.fund >=2), 'fund_r'] = 0

In [95]:
#Recode frequency of prayer to dummy. Reference category =  "less than once per day"
gss["pray_r"]=gss["pray"]
gss.loc[(gss.pray <=2), 'pray_r'] = 1
gss.loc[(gss.pray >2), 'pray_r'] = 0

In [96]:
#Recode religious affiliation to be consistent with the RELTRAD taxonomy devleoped by Steensland et al (2000), as coded in an updated more=accurate version for the GSS in 2015 by Ryan Burge (https://github.com/ryanburge/reltrad)

#Categories
# 1 evangelical
# 2 mainline
# 3 black protestant
# 4 catholic
# 5 jewish
# 6 other
# 7 nonaffiliated

##create xaffil variable to store recoded religion variable values
gss["xaffil"]=gss["relig"]
gss.loc[(gss.relig ==2), 'xaffil'] = 4
gss.loc[(gss.relig ==3), 'xaffil'] = 5
gss.loc[(gss.relig ==4), 'xaffil'] = 9
gss.loc[(gss.relig >=5) & (gss.relig <=10), 'xaffil'] = 6
gss.loc[(gss.relig ==12), 'xaffil'] = 6
gss.loc[(gss.relig ==13), 'xaffil'] = 1

##create dummy variable to identify black protestant respondants
gss["xbp"]=gss["other"]
bp_list=[7, 14, 15, 21, 37, 38, 56, 78, 79, 85, 86, 87, 88, 98, 103, 104, 128, 133]
bpd_list= [12,13,20,21]
###first identifying black protestant responses from "other religions" variable
for value in bp_list:
    gss.loc[(gss.other ==value), 'xbp'] = 1
 ###next identifying black protestant responses from "denomination" variable   
for value in bpd_list:
    gss.loc[(gss.denom ==value), 'xbp'] = 1
###next, using race of respondant for ambiguous denominations 
gss["bl"]=gss["race"]
gss.loc[(gss.race ==2), 'bl']=1
gss.loc[(gss.race ==3) | (gss.race ==1), 'bl']=0
gss["bldenom"]=gss["denom"]*gss["bl"]

bldenom_list=[23,28,18,15,10,11,14]
for value in bldenom_list:
    gss.loc[(gss.bldenom ==value), 'xbp']=1
###finally, by pairing race of respondant with ambiguous "other religion"
gss["blother"]=gss["other"]*gss["bl"]
gss.loc[(gss.blother ==93), 'xbp']=1

##create dummy variable to identify evangelical respondants
###first by identifying obvious denominations
gss["xev"]=gss["other"]
xev_list=[2, 3, 5, 6, 9, 10, 12, 13, 16, 18, 20, 22, 24, 26, 27, 28, 31, 32, 34, 35, 36, 39, 41, 42, 43, 45, 47, 51, 52, 53, 55, 57, 63, 65, 66, 67, 68, 69, 76, 77, 83, 84, 90, 91, 92, 94, 97, 100, 101, 102, 106, 107, 108, 109, 110, 111, 112, 115, 116, 117, 118, 120, 121, 122, 124, 125, 127, 129, 131, 132, 134, 135, 138, 139, 140, 146]
for value in xev_list:
    gss.loc[(gss.xev ==value), 'xev'] = 1
gss.loc[(gss.xev !=1), 'xev'] = 0
denom_list=[32,33,34,42]
for value in denom_list:
    gss.loc[(gss.denom ==value), 'xev'] = 1
###next using respondants race
gss["wh"]=gss["race"]
gss.loc[(gss.race ==2), 'wh'] = 0
gss.loc[(gss.race ==3), 'wh'] = 1
gss["whdenom"]=gss["denom"]*gss["wh"]
wh_list=[23,18,15,10,14]
for value in wh_list:
    gss.loc[(gss.whdenom ==value), 'xev'] = 1
gss["whother"]=gss["other"]*gss["wh"]
gss.loc[(gss.whother ==93), 'xev'] = 1
gss.loc[(gss.xbp ==1), 'xev'] = 0

##create dummy variable to identify mainline respondants
gss["xml"]=gss["other"]
ml_list=[1, 8, 19, 23, 25, 40, 44, 46, 48, 49, 50, 54, 70, 71, 72, 73, 81, 89, 96, 99, 105, 119, 148]
for value in ml_list:
    gss.loc[(gss.xml ==value), 'xml'] = 1
gss.loc[(gss.xml !=1), 'xml'] = 0
mld_list=[22,30,31,35,38,40,41,43,48,50]
for value in mld_list:
    gss.loc[(gss.denom ==value), 'xml'] = 1
gss.loc[(gss.whdenom ==11) | (gss.whdenom ==28), 'xml']=1

##create dummy variable to identify catholic respondants
gss["xcath"]=gss["other"]
gss.loc[(gss.xaffil ==4), 'xcath'] = 1

##create dummy variable to identify jewish respondants
gss["xjew"]=gss["other"]
gss.loc[(gss.xaffil ==5), 'xjew'] = 1

##create dummy variable to identify other respondants
gss["xother"]=gss["other"]
other_list=[11, 17, 29, 30, 33, 58, 59, 60, 61, 62, 64, 74, 75, 80, 82, 95, 113, 114, 130, 136, 141, 145]
for value in other_list:
    gss.loc[(gss.other ==value), 'xother'] = 1
gss.loc[(gss.xother !=1), 'xother'] = 0

gss["noxev"]=1-gss["xev"]
gss["noxevxaf"]=gss["noxev"]*gss["xaffil"]
gss.loc[(gss.noxevxaf==6), 'xother'] = 1

gss["xnonaff"]=1-gss["xaffil"]
gss.loc[(gss.xnonaff==9), 'xnonaff'] = 1
gss.loc[(gss.noxevxaf!=1), 'xnonaff'] = 0

##classify non-denomonational people as evangelical if they attend church at least once per month (I *think* this is what the original code is doing, but I'm not certain. Will check later after re-reading Steensland et al.The code below simply copies the function of the original code exactly but with python syntax.)
gss["xprotdk"]=gss["denom"]
dk_list=[0,1,2,3,9]
for value in dk_list:
    gss.loc[(gss.attend ==value), 'xprotdk'] = 0
gss.loc[(gss.xprotdk ==1), 'xev'] = 1

#generate the final reltrad variable using all dummies to ID values
gss["reltrad"]=0
gss.loc[(gss.xnonaff ==1), 'reltrad'] = 7
gss.loc[(gss.xother ==1), 'reltrad'] = 6
gss.loc[(gss.xjew ==1), 'reltrad'] = 5
gss.loc[(gss.xcath ==1), 'reltrad'] = 4
gss.loc[(gss.xbp ==1), 'reltrad'] = 3
gss.loc[(gss.xml ==1), 'reltrad'] = 2
gss.loc[(gss.xev ==1), 'reltrad'] = 1
gss.loc[(gss.reltrad ==0), 'reltrad'] = None


In [97]:
#Create source of anger variable; in past week, who was the respondant angry with.

#Categories: 
# 0 "none"
# 1 "family" 
# 2 "friend" 
# 3 "work" 
# 4 "self" 
# 5 "other"

##Family
gss["angryfamily"]=0
gss.loc[(gss.madat1 ==1), 'angryfamily'] = 1
gss.loc[(gss.madat2 ==1), 'angryfamily'] = 1
gss.loc[(gss.madat3 ==1), 'angryfamily'] = 1
gss.loc[(gss.madat4 ==1), 'angryfamily'] = 1
gss.loc[(gss.madat5 ==1), 'angryfamily'] = 1
gss.loc[(gss.madat6 ==1), 'angryfamily'] = 1
gss.loc[(gss.madat7 ==1), 'angryfamily'] = 1
gss.loc[(gss.madat8 ==1), 'angryfamily'] = 1
gss.loc[(gss.madat23 ==1), 'angryfamily'] = 1
gss.loc[(gss.madat23 ==2), 'angryfamily'] = 1

##Friend
gss["angryfriend"]=0
gss.loc[(gss.madat9 ==1), 'angryfriend'] = 1
gss.loc[(gss.madat10 ==1), 'angryfriend'] = 1
gss.loc[(gss.madat11 ==1), 'angryfriend'] = 1

##Work
gss["angrywork"]=0
gss.loc[(gss.madat12 ==1), 'angrywork'] = 1
gss.loc[(gss.madat13 ==1), 'angrywork'] = 1
gss.loc[(gss.madat14 ==1), 'angrywork'] = 1
gss.loc[(gss.madat15 ==1), 'angrywork'] = 1
gss.loc[(gss.madat16 ==1), 'angrywork'] = 1

##Self
gss["angryself"]=0
gss.loc[(gss.madat21 ==1), 'angryself'] = 1

##Other
gss["angryother"]=0
gss.loc[(gss.madat17 ==1), 'angryother'] = 1
gss.loc[(gss.madat18 ==1), 'angryother'] = 1
gss.loc[(gss.madat19 ==1), 'angryother'] = 1
gss.loc[(gss.madat20 ==1), 'angryother'] = 1
gss.loc[(gss.madat22 ==1), 'angryother'] = 1
gss.loc[(gss.madat23 >3), 'angryother'] = 1

gss["angrysource"]=0
gss.loc[(gss.angryfamily ==1), 'angrysource'] = 1
gss.loc[(gss.angryfriend ==1), 'angrysource'] = 2
gss.loc[(gss.angrywork ==1), 'angrysource'] = 3
gss.loc[(gss.angryself ==1), 'angrysource'] = 4
gss.loc[(gss.angryother ==1), 'angrysource'] = 5
gss.loc[(gss.madat23 == None), 'angrysource'] = 0

In [98]:
#Interaction Variables
#These variables allow us to discern whether interaction effects are relevant in this model

##sex interactions
gss["sexage"]=gss["sex_r"]*gss["age_c"]
gss["sexrace"]=gss["sex_r"]*gss["race_r"]
gss["sexcollege"]=gss["sex_r"]*gss["degree_r"]
gss["sexregion"]=gss["sex_r"]*gss["region_r"]
gss["sexmarital"]=gss["sex_r"]*gss["marital_r"]
gss["sexchilds"]=gss["sex_r"]*gss["childs_r"]
gss["sexfund"]=gss["sex_r"]*gss["fund_r"]
gss["sexpray"]=gss["sex_r"]*gss["pray_r"]
gss["sexreligion"]=gss["sex_r"]*gss["reltrad"]
gss["sexattend"]=gss["sex_r"]*gss["attend"]
gss["sexhowangry"]=gss["sex_r"]*gss["howangry"]
gss["sexangrysource"]=gss["sex_r"]*gss["angrysource"]

##race interactions
gss["raceage"]=gss["race_r"]*gss["age_c"]
gss["racecollege"]=gss["race_r"]*gss["degree_r"]
gss["raceregion"]=gss["race_r"]*gss["region_r"]
gss["racemarital"]=gss["race_r"]*gss["marital_r"]
gss["racechilds"]=gss["race_r"]*gss["childs_r"]
gss["racefund"]=gss["race_r"]*gss["fund_r"]
gss["racepray"]=gss["race_r"]*gss["pray_r"]
gss["racereligion"]=gss["race_r"]*gss["reltrad"]
gss["raceattend"]=gss["race_r"]*gss["attend"]
gss["racehowangry"]=gss["race_r"]*gss["howangry"]
gss["raceangrysource"]=gss["race_r"]*gss["angrysource"]

##degree interactions
gss["degreeage"]=gss["degree_r"]*gss["age_c"]
gss["degreeregion"]=gss["degree_r"]*gss["region_r"]
gss["degreemarital"]=gss["degree_r"]*gss["marital_r"]
gss["degreechilds"]=gss["degree_r"]*gss["childs_r"]
gss["degreefund"]=gss["degree_r"]*gss["fund_r"]
gss["degreepray"]=gss["degree"]*gss["pray_r"]
gss["degreereligion"]=gss["degree_r"]*gss["reltrad"]
gss["degreeattend"]=gss["degree_r"]*gss["attend"]
gss["degreehowangry"]=gss["degree_r"]*gss["howangry"]
gss["degreeangrysource"]=gss["degree_r"]*gss["angrysource"]

##age interactions
gss["ageregion"]=gss["age_c"]*gss["region_r"]
gss["agemarital"]=gss["age_c"]*gss["marital_r"]
gss["agechilds"]=gss["age_c"]*gss["childs_r"]
gss["agefund"]=gss["age_c"]*gss["fund_r"]
gss["agepray"]=gss["age"]*gss["pray_r"]
gss["agereligion"]=gss["age_c"]*gss["reltrad"]
gss["ageattend"]=gss["age_c"]*gss["attend"]
gss["agehowangry"]=gss["age_c"]*gss["howangry"]
gss["ageangrysource"]=gss["age_c"]*gss["angrysource"]

##region interactions
gss["regionmarital"]=gss["region_r"]*gss["marital_r"]
gss["regionchilds"]=gss["region_r"]*gss["childs_r"]
gss["regionfund"]=gss["region_r"]*gss["fund_r"]
gss["regionpray"]=gss["region"]*gss["pray_r"]
gss["regionreligion"]=gss["region_r"]*gss["reltrad"]
gss["regionattend"]=gss["region_r"]*gss["attend"]
gss["regionhowangry"]=gss["region_r"]*gss["howangry"]
gss["regionangrysource"]=gss["region_r"]*gss["angrysource"]

##marital interactions
gss["maritalchilds"]=gss["marital_r"]*gss["childs_r"]
gss["maritalfund"]=gss["marital_r"]*gss["fund_r"]
gss["maritalpray"]=gss["marital"]*gss["pray_r"]
gss["maritalreligion"]=gss["marital_r"]*gss["reltrad"]
gss["maritalattend"]=gss["marital_r"]*gss["attend"]
gss["maritalhowangry"]=gss["marital_r"]*gss["howangry"]
gss["maritalangrysource"]=gss["marital_r"]*gss["angrysource"]

##childs interactions
gss["childsfund"]=gss["childs_r"]*gss["fund_r"]
gss["childspray"]=gss["childs"]*gss["pray_r"]
gss["childsreligion"]=gss["childs_r"]*gss["reltrad"]
gss["childsattend"]=gss["childs_r"]*gss["attend"]
gss["childshowangry"]=gss["childs_r"]*gss["howangry"]
gss["childsangrysource"]=gss["childs_r"]*gss["angrysource"]

##fundamentalist interactions
gss["fundfund"]=gss["fund_r"]*gss["fund_r"]
gss["fundpray"]=gss["fund"]*gss["pray_r"]
gss["fundreligion"]=gss["fund_r"]*gss["reltrad"]
gss["fundattend"]=gss["fund_r"]*gss["attend"]
gss["fundhowangry"]=gss["fund_r"]*gss["howangry"]
gss["fundangrysource"]=gss["fund_r"]*gss["angrysource"]

#pray interactions
gss["prayreligion"]=gss["pray_r"]*gss["reltrad"]
gss["prayattend"]=gss["pray_r"]*gss["attend"]
gss["prayhowangry"]=gss["pray_r"]*gss["howangry"]
gss["prayangrysource"]=gss["pray_r"]*gss["angrysource"]

##religion interactions
gss["religionattend"]=gss["attend"]*gss["reltrad"]
gss["religionhowangry"]=gss["reltrad"]*gss["howangry"]
gss["attendangrysource"]=gss["reltrad"]*gss["angrysource"]

##attend interactions
gss["attendhowangry"]=gss["attend"]*gss["howangry"]
gss["attendangrysource"]=gss["attend"]*gss["angrysource"]

##howangry interactions
gss["howangryangrysource"]=gss["howangry"]*gss["angrysource"]



In [99]:
reltrad_db=gss.to_stata("reltrad_data.dta")
converted_pd = pd.io.stata.read_stata('reltrad_data.dta', convert_categoricals=True)

In [100]:
#Imputing missing values
########################

#Dataset with means of each column imputed


In [ ]:
gss_complete=pd.DataFrame()

In [109]:
#Impute the modes for the categorical varaibles by
#1.bringing in a dataset with categorical variables labeled as strings

gss_modes=pd.read_csv("GSS_1996.csv")
for varname in gss_modes:
    try:
        mean=gss_modes[f"{varname}"].mean() #if the columns are not numeric, this will fail.
    except:
        mode=gss_modes[f"{varname}"].mode()
        gss_modes.loc[(gss_modes[f"{varname}"].isnull()), varname] = mode
        gss_complete.append(gss_modes[f"{varname}"])
#2.comparing this dataset to the purely numeric one, identifying string variables
#3.Creating a dataframe to store variables with modes imputed
#3.imputing the mode for these variables only in the new dataframe

In [110]:
gss_complete

""


In [47]:
#Create a dataframe with the means imputed for every missing value
gss_means=gss.copy(deep=True)
for varname in gss_means:
    mean=gss_means[varname].mean()
    gss_means.loc[(gss_means[varname].isnull()), varname] = mean

In [1]:
from fancyimpute import KNN, NuclearNormMinimization, SoftImpute, BiScaler

In [6]:
X_filled_knn = KNN(k=3).fit_transform(gss)

Imputing row 1/2904 with 616 missing, elapsed time: 100.518
Imputing row 101/2904 with 604 missing, elapsed time: 102.887
Imputing row 201/2904 with 602 missing, elapsed time: 104.709
Imputing row 301/2904 with 566 missing, elapsed time: 106.560
Imputing row 401/2904 with 562 missing, elapsed time: 108.795
Imputing row 501/2904 with 661 missing, elapsed time: 110.737
Imputing row 601/2904 with 698 missing, elapsed time: 112.995
Imputing row 701/2904 with 709 missing, elapsed time: 116.214
Imputing row 801/2904 with 645 missing, elapsed time: 119.876
Imputing row 901/2904 with 610 missing, elapsed time: 122.252
Imputing row 1001/2904 with 584 missing, elapsed time: 124.302
Imputing row 1101/2904 with 603 missing, elapsed time: 126.376
Imputing row 1201/2904 with 608 missing, elapsed time: 128.514
Imputing row 1301/2904 with 584 missing, elapsed time: 130.728
Imputing row 1401/2904 with 590 missing, elapsed time: 132.950
Imputing row 1501/2904 with 846 missing, elapsed time: 134.979
Impu